# Run XGBoost Model<br>
Steps:<br>
1. Read dataset file
2. Create training & test datasets
3. Create & run XGBoost model
4. Evaluation
5. Save output into file

# Step 1. Read Dataset File

## Import libraries

In [289]:
# Import file and data libraries
import math
import pickle
import numpy as np
import pandas as pd
from collections import Counter

# Import Matthews Correlation Coefficient module (MCC)
from sklearn.metrics import matthews_corrcoef

# Import libraries
import xgboost as XGB

## Read dataset file

In [290]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [291]:
# Read X & Y scaled datasets from Pickle files
dataset_path = "/content/drive/My Drive/Colab Notebooks/3. Code/3. Compare/Dataset/"

with open(dataset_path + "Dataset V2.pickle", "rb") as f:
    X = pickle.load(f)  # Contains Date, HL_Avg, Volume, HL_Avg_Norm, Volume_Norm
    Y = pickle.load(f)  # Contains Y_Avg, X_Last, Y_Diff, Y_Band, Y_Gap, Y_Days, Y_Avg_Norm, X_Last_Norm, Y_Band_Norm
    Metadata = pickle.load(f) # Contains the Company ID, ticker, sector and Dataset ID
f.close()

# Step 2. Create training & test datasets

## Make X & Y uniform length

In [292]:
# The min length of X datasets is 1230: min([len(X[i]) for i in range(len(X))]) 
# Make X has uniform length by taking the last 1230 values: (take the normalised price and volume)
# X[0][-1230:] means taking the last 1230 values
# Use 1 variable (price only) for now. Only take 100 days
X_Uniform_Length = [X[i][-50:]["Volume_Norm"] for i in range(len(X))]
# X_Uniform_Length = [X[i][-1230:][["HL_Avg_Norm","Volume_Norm"]] for i in range(len(X))]

# Convert to Numpy array 
X_XGB = np.array([np.array(X_Uniform_Length[i]) for i in range(len(X_Uniform_Length))])
X_XGB.shape

(3989, 50)

In [293]:
# Make Price and volume in 1 variable (stack them side by side)
#X_XGB = np.hstack((X_XGB[:,:,0], X_XGB[:,:100,1]))
X_XGB.shape

(3989, 50)

In [294]:
# This is XGB with the output (Y) is categorical. So use the Y Band, not the price.
Y_XGB_Band = np.array([Y[i][3] for i in range(len(Y))]) # Y[i][3] is the Y Band
print(Y_XGB_Band.shape, Y_XGB_Band[0])

(3989,) U1


## Create functions

In [295]:
# Convert Y from price bands (U1, U2, D1, ...) to class ID (0 to 9)
Band_List = ["D5","D4","D3","D2","D1","U1","U2","U3","U4","U5"]
print(Band_List[0], Band_List.index("D5"))

def Convert_Band_To_Class(Band):
    return Band_List.index(Band)

print(Convert_Band_To_Class("U1"))

D5 0
5


In [296]:
# Convert Y from class ID (0 to 9) to price bands (U1, U2, D1, ...)
def Convert_Class_To_Band(Class):
    return Band_List[int(Class)]

print(Convert_Class_To_Band(3.0))

D2


In [297]:
# Convert Y from price band to class ID (0 to 9)
Y_XGB = np.array([Convert_Band_To_Class(Y_XGB_Band[i]) for i in range(len(Y_XGB_Band))])
Y_XGB.shape

(3989,)

## Split into training and test

In [298]:
# Split X & Y into Training and Test dataasets
# Test dataset: the first 1000, Training dataset: the remaining (3000 - 11 = 2989)
X_Train = X_XGB[1000:]
X_Test  = X_XGB[:1000]
Y_Train = Y_XGB[1000:]
Y_Test  = Y_XGB[:1000]
print(X_Train.shape, X_Test.shape, Y_Train.shape, Y_Test.shape)

(2989, 50) (1000, 50) (2989,) (1000,)


# Step 3. Create & run XGBoost Model

## Create XGBoost model

In [299]:
File_Name = "E150 V50 Softmax.csv"

In [300]:
# Create XGB model
# Use 100 estimator first (increase to 1000 later)
model = XGB.XGBRegressor(objective="multi:softmax", n_estimators=150, num_class=10) #, gamma=0.1
#max_depth=4, min_child_weight=0.5, subsample=0.75, eta=0.4, reg_lambda=0.9, reg_alpha=0.01

In [301]:
# Fit the training data
model.fit(X_Train, Y_Train)

XGBRegressor(n_estimators=150, num_class=10, objective='multi:softmax')

## Forecast the price band

In [302]:
# Forecast the test data
F = model.predict(X_Test)
# The output is the class ID i.e. 0 to 9 (but float, i.e. 9.0 and 5.0)
F[:50]

array([6., 6., 7., 6., 5., 7., 6., 5., 6., 5., 6., 5., 4., 9., 8., 6., 4.,
       6., 5., 6., 7., 6., 9., 6., 4., 6., 9., 5., 5., 6., 5., 6., 4., 7.,
       5., 5., 6., 6., 6., 6., 7., 6., 6., 9., 3., 6., 4., 4., 6., 5.],
      dtype=float32)

In [303]:
# Convert the forecast result (F) back to price band
F_Band_List = []
for DS in range(len(F)):
    F_Band = Convert_Class_To_Band(F[DS])
    F_Band_List.append(F_Band)
print(F_Band_List)

['U2', 'U2', 'U3', 'U2', 'U1', 'U3', 'U2', 'U1', 'U2', 'U1', 'U2', 'U1', 'D1', 'U5', 'U4', 'U2', 'D1', 'U2', 'U1', 'U2', 'U3', 'U2', 'U5', 'U2', 'D1', 'U2', 'U5', 'U1', 'U1', 'U2', 'U1', 'U2', 'D1', 'U3', 'U1', 'U1', 'U2', 'U2', 'U2', 'U2', 'U3', 'U2', 'U2', 'U5', 'D2', 'U2', 'D1', 'D1', 'U2', 'U1', 'U4', 'U2', 'U5', 'U1', 'U1', 'U2', 'U1', 'D1', 'U2', 'U2', 'D3', 'D2', 'U2', 'U1', 'D3', 'U1', 'U1', 'U3', 'D1', 'D1', 'U5', 'D2', 'U1', 'U3', 'U2', 'U2', 'U2', 'D2', 'U3', 'U3', 'U5', 'D1', 'D1', 'U1', 'U2', 'U2', 'U3', 'U3', 'U2', 'D1', 'U1', 'U2', 'U2', 'D1', 'U2', 'U1', 'U1', 'U1', 'U1', 'D2', 'U1', 'U1', 'D1', 'U2', 'U3', 'D1', 'D2', 'U4', 'U1', 'D1', 'D1', 'U2', 'U2', 'U1', 'D1', 'D3', 'D1', 'U1', 'D1', 'U3', 'U1', 'D1', 'U2', 'U1', 'U1', 'U2', 'U1', 'D1', 'U4', 'U1', 'D1', 'U1', 'U1', 'D1', 'U1', 'U1', 'U1', 'D1', 'U3', 'U1', 'U5', 'U2', 'U5', 'U5', 'D1', 'U3', 'U1', 'U1', 'U1', 'U2', 'U1', 'U2', 'U2', 'U3', 'U2', 'U1', 'U1', 'U1', 'D2', 'U1', 'U5', 'U1', 'U1', 'U1', 'U1', 'U1', 'U2

# Step 4. Evaluation

In [304]:
# Y_Test is also the class ID ,i.e. number 0 to 9
print(Y_Test[:100])

[5 2 5 9 9 2 9 8 6 5 3 7 4 3 9 0 0 7 7 5 0 5 9 4 1 7 2 9 0 9 5 7 9 8 0 9 7
 9 5 2 6 3 8 8 9 9 9 1 4 0 9 1 9 4 9 9 1 9 7 8 9 9 9 0 5 3 9 9 4 8 9 0 8 8
 2 4 6 9 9 5 9 2 9 8 4 3 9 9 3 7 4 5 7 9 9 1 6 9 4 9]


In [305]:
# Convert Y_Test from class ID to price band
Y_Band_List = []
for DS in range(len(Y_Test)):
    Y_Band = Convert_Class_To_Band(Y_Test[DS])
    Y_Band_List.append(Y_Band)
print(Y_Band_List)

['U1', 'D3', 'U1', 'U5', 'U5', 'D3', 'U5', 'U4', 'U2', 'U1', 'D2', 'U3', 'D1', 'D2', 'U5', 'D5', 'D5', 'U3', 'U3', 'U1', 'D5', 'U1', 'U5', 'D1', 'D4', 'U3', 'D3', 'U5', 'D5', 'U5', 'U1', 'U3', 'U5', 'U4', 'D5', 'U5', 'U3', 'U5', 'U1', 'D3', 'U2', 'D2', 'U4', 'U4', 'U5', 'U5', 'U5', 'D4', 'D1', 'D5', 'U5', 'D4', 'U5', 'D1', 'U5', 'U5', 'D4', 'U5', 'U3', 'U4', 'U5', 'U5', 'U5', 'D5', 'U1', 'D2', 'U5', 'U5', 'D1', 'U4', 'U5', 'D5', 'U4', 'U4', 'D3', 'D1', 'U2', 'U5', 'U5', 'U1', 'U5', 'D3', 'U5', 'U4', 'D1', 'D2', 'U5', 'U5', 'D2', 'U3', 'D1', 'U1', 'U3', 'U5', 'U5', 'D4', 'U2', 'U5', 'D1', 'U5', 'D3', 'D4', 'D5', 'D3', 'U2', 'U4', 'U3', 'U4', 'U3', 'U5', 'D5', 'U4', 'D5', 'U4', 'U5', 'D4', 'D4', 'U5', 'U5', 'D2', 'D4', 'U5', 'D2', 'U4', 'U5', 'D1', 'U3', 'U4', 'U2', 'U5', 'D3', 'D2', 'U5', 'U5', 'U2', 'U1', 'U5', 'D5', 'U5', 'U3', 'D5', 'D5', 'D2', 'U5', 'U3', 'U2', 'D3', 'U4', 'D2', 'D4', 'U1', 'U1', 'U5', 'U5', 'U3', 'U5', 'D2', 'D2', 'D3', 'U4', 'U4', 'U4', 'U4', 'D1', 'U5', 'U1', 'D5

In [306]:
# Find out how many forecasts are correct
Status_List = []
for i in range(len(Y_Band_List)):
    Status = 1 if Y_Band_List[i] == F_Band_List[i] else 0
    Status_List.append(Status)
sum(Status_List)

130

In [307]:
# Get the count of each band
c = Counter(F_Band_List)
print(c.keys(), c.values())

dict_keys(['U2', 'U3', 'U1', 'D1', 'U5', 'U4', 'D2', 'D3', 'D5', 'D4']) dict_values([281, 98, 299, 153, 90, 32, 39, 5, 1, 2])


In [308]:
# Calculate MCC
matthews_corrcoef(Y_Band_List, F_Band_List) # 0.0765850700912446 (for Estimator=150 Price=200days)

0.00844096918151303

# Step 5. Save output to CSV file

In [309]:
Output_File = ""
for i in range(len(Y_Band_List)):
    output = Y_Band_List[i] + "," +  F_Band_List[i] + "," + str(Status_List[i]) + "\n"
    Output_File = Output_File + output   

In [310]:
# Save to CSV file
File_Path = "/content/drive/My Drive/Colab Notebooks/3. Code/3. Compare/Output/6. XGBoost/"
Full_Path = File_Path + File_Name
print(Full_Path)
f = open(Full_Path, "w")
f.write(Output_File)
f.close()

/content/drive/My Drive/Colab Notebooks/3. Code/3. Compare/Output/6. XGBoost/E150 V50 Softmax.csv
